## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-20-22-52-14 +0000,nyt,A.I. Is Coming for the 2026 Midterms,https://www.nytimes.com/2026/02/20/us/politics...
1,2026-02-20-22-52-00 +0000,wsj,Opinion | How George Harrison Transformed the ...,https://www.wsj.com/opinion/how-george-harriso...
2,2026-02-20-22-50-00 +0000,cbc,Where does the U.S. Supreme Court voiding some...,https://www.cbc.ca/news/politics/supreme-court...
3,2026-02-20-22-47-51 +0000,nypost,Colombian illegal migrant learns fate for stra...,https://nypost.com/2026/02/20/us-news/colombia...
4,2026-02-20-22-47-35 +0000,nypost,California first responders rescue camper and ...,https://nypost.com/2026/02/20/us-news/californ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2525/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,79
9,court,38
8,supreme,36
12,tariffs,31
63,tariff,24


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
49,2026-02-20-21-44-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...,333
124,2026-02-20-18-15-00 +0000,wsj,The Supreme Court decision to strike down Pres...,https://www.wsj.com/economy/trade/trump-tariff...,255
66,2026-02-20-20-56-39 +0000,wapo,"Trump imposes new tariffs, denounces Supreme C...",https://www.washingtonpost.com/national-securi...,240
152,2026-02-20-17-23-09 +0000,nypost,Democrats giddy over Trump Supreme Court tarif...,https://nypost.com/2026/02/20/us-news/democrat...,228
122,2026-02-20-18-37-16 +0000,nypost,Trump goes on blistering attack against Suprem...,https://nypost.com/2026/02/20/us-news/trump-ri...,226


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
49,333,2026-02-20-21-44-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...
124,78,2026-02-20-18-15-00 +0000,wsj,The Supreme Court decision to strike down Pres...,https://www.wsj.com/economy/trade/trump-tariff...
246,69,2026-02-20-11-27-40 +0000,nypost,Inside covert police operation to arrest ex-Pr...,https://nypost.com/2026/02/20/world-news/insid...
190,66,2026-02-20-15-40-27 +0000,48hills,Screen Grabs: End of days at Fear and Faith Ho...,https://48hills.org/2026/02/screen-grabs-end-o...
273,56,2026-02-20-08-46-25 +0000,wsj,Trump Says He Will Impose 10% Tariff Under Dif...,https://www.wsj.com/livecoverage/stock-market-...
228,54,2026-02-20-13-00-00 +0000,wsj,After President Trump’s first year back in off...,https://www.wsj.com/economy/trade/tariffs-comm...
184,51,2026-02-20-15-58-01 +0000,nypost,The government agency relying on Wall Street t...,https://nypost.com/2026/02/20/business/develop...
341,35,2026-02-20-00-08-02 +0000,nypost,Husband of embattled Labor Secretary Lori Chav...,https://nypost.com/2026/02/19/us-news/husband-...
110,35,2026-02-20-19-18-02 +0000,nypost,"As US economy drastically slows, Fed’s preferr...",https://nypost.com/2026/02/20/business/as-us-e...
279,33,2026-02-20-07-18-00 +0000,wsj,Air Liquide Posts Higher Profit on Sales Growt...,https://www.wsj.com/business/earnings/air-liqu...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
